# Few-Shot Templates

- Author: [hong-seongmin](https://github.com/hong-seongmin)
- Design: 
- Peer Review: [Wooseok-Jeong](https://github.com/jeong-wooseok), [Hye-yoon](https://github.com/Hye-yoonJeong)
- This is a part of [LangChain OpenTutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/sub-graph.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239937-lesson-2-sub-graphs)


## Overview

LangChain's Few-Shot Prompting provides a robust framework for guiding language models to generate high-quality outputs by supplying carefully selected examples. This technique minimizes the need for extensive model fine-tuning while ensuring precise, context-aware results across diverse use cases.

### Key Components:

- **Few-Shot Prompt Templates**: Define the structure and format of prompts by embedding illustrative examples, guiding the model to produce consistent outputs.
- **Example Selection Strategies**: Dynamically select the most relevant examples for a given query, enhancing the model's contextual understanding and response accuracy.
- **Chroma Vector Store**: A powerful utility for storing and retrieving examples based on semantic similarity, enabling scalable and efficient prompt construction.

---

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [FewShotPromptTemplate](#FewShotPromptTemplate)
- [Dynamic Example Selection with Chroma](#dynamic-example-selection-with-chroma)
- [FewShotChatMessagePromptTemplate](#FewShotChatMessagePromptTemplate)

---

### References

- [LangChain Documentation: Few-shot prompting](https://python.langchain.com/docs/concepts/few_shot_prompting/#3-selecting-examples)
- [How to better prompt when doing SQL question-answering](https://python.langchain.com/docs/how_to/sql_prompting/#few-shot-examples)

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
!pip install langchain-opentutorial

    numpy (>=1.19.*) ; python_version >= "3.7"
           ~~~~~~~^


In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_openai",
        "langchain-chroma",
    ],
    verbose=False,
    upgrade=False,
)

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "09-FewShot-Prompt-Templates",
    }
)

Environment variables have been set successfully.


## FewShotPromptTemplate

Few-shot prompting is a powerful technique that guides language models to produce accurate and contextually relevant outputs by providing a small set of carefully designed examples. LangChain's `FewShotPromptTemplate` streamlines this process, allowing users to construct flexible and reusable prompts for various tasks like question answering, summarization, and correction.

1. **Designing Few-Shot Prompts**

   - Define examples that illustrate the desired structure and style of the output.
   - Ensure examples cover edge cases to improve model understanding and performance.

2. **Dynamic Example Selection**

   - Use semantic similarity techniques or vector-based search to select the most relevant examples for a query.

3. **Integrating Few-Shot Prompts**

   - Combine prompt templates with language models to create robust chains for generating responses.

### FewShotPromptTemplate Example

The FewShotPromptTemplate allows you to provide a language model with a small set of examples that demonstrate the desired structure and format of its output. By leveraging these examples, the model can better understand the context and generate more accurate responses for new queries. This technique is especially useful for tasks like question answering, summarization, or generating structured outputs.

Below, we define a few examples to help the model answer questions more effectively by breaking them down into intermediate steps. We then use the `FewShotPromptTemplate` to format the prompt dynamically based on the query.

---

In [5]:
from langchain_openai import ChatOpenAI

# Initialize the language model
llm = ChatOpenAI(
    temperature=0,  # Creativity
    model_name="gpt-4o-mini",  # Use a valid model name
)

# User query
question = "What is the capital of United States of America?"

# Query the model
response = llm.predict(question)

# Print the response
print(response)

The capital of the United States of America is Washington, D.C.


In [6]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# Define examples for the few-shot prompt
examples = [
    {
        "question": "Who lived longer, Steve Jobs or Einstein?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein
""",
    },
    {
        "question": "When was the founder of Naver born?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is the founder of Naver?
Intermediate Answer: Naver was founded by Lee Hae-jin.
Additional Question: When was Lee Hae-jin born?
Intermediate Answer: Lee Hae-jin was born on June 22, 1967.
The final answer is: June 22, 1967
""",
    },
    {
        "question": "Who was the reigning king when Yulgok Yi's mother was born?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is Yulgok Yi's mother?
Intermediate Answer: Yulgok Yi's mother is Shin Saimdang.
Additional Question: When was Shin Saimdang born?
Intermediate Answer: Shin Saimdang was born in 1504.
Additional Question: Who was the king of Joseon in 1504?
Intermediate Answer: The king of Joseon in 1504 was Yeonsangun.
The final answer is: Yeonsangun
""",
    },
    {
        "question": "Are the directors of Oldboy and Parasite from the same country?",
        "answer": """Does this question require additional questions: Yes.
Additional Question: Who is the director of Oldboy?
Intermediate Answer: The director of Oldboy is Park Chan-wook.
Additional Question: Which country is Park Chan-wook from?
Intermediate Answer: Park Chan-wook is from South Korea.
Additional Question: Who is the director of Parasite?
Intermediate Answer: The director of Parasite is Bong Joon-ho.
Additional Question: Which country is Bong Joon-ho from?
Intermediate Answer: Bong Joon-ho is from South Korea.
The final answer is: Yes
""",
    },
]

In [7]:
# Create an example prompt template
example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

# Print the first formatted example
print(example_prompt.format(**examples[0]))

Question:
Who lived longer, Steve Jobs or Einstein?
Answer:
Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein



In [9]:
# Initialize the FewShotPromptTemplate
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# Example question
question = "How old was Bill Gates when Google was founded?"

# Generate the final prompt
final_prompt = few_shot_prompt.format(question=question)
print(final_prompt)

Question:
Who lived longer, Steve Jobs or Einstein?
Answer:
Does this question require additional questions: Yes.
Additional Question: At what age did Steve Jobs die?
Intermediate Answer: Steve Jobs died at the age of 56.
Additional Question: At what age did Einstein die?
Intermediate Answer: Einstein died at the age of 76.
The final answer is: Einstein


Question:
When was the founder of Naver born?
Answer:
Does this question require additional questions: Yes.
Additional Question: Who is the founder of Naver?
Intermediate Answer: Naver was founded by Lee Hae-jin.
Additional Question: When was Lee Hae-jin born?
Intermediate Answer: Lee Hae-jin was born on June 22, 1967.
The final answer is: June 22, 1967


Question:
Who was the reigning king when Yulgok Yi's mother was born?
Answer:
Does this question require additional questions: Yes.
Additional Question: Who is Yulgok Yi's mother?
Intermediate Answer: Yulgok Yi's mother is Shin Saimdang.
Additional Question: When was Shin Saimdang bo

In [10]:
# Query the model with the final prompt
response = llm.predict(final_prompt)
print(response)

Does this question require additional questions: Yes.  
Additional Question: When was Google founded?  
Intermediate Answer: Google was founded on September 4, 1998.  
Additional Question: What is Bill Gates' birth date?  
Intermediate Answer: Bill Gates was born on October 28, 1955.  
Additional Question: How old was Bill Gates on September 4, 1998?  
Intermediate Answer: Bill Gates was 42 years old when Google was founded.  
The final answer is: 42 years old.


## Dynamic Example Selection with Chroma

When working with multiple examples, selecting the most relevant ones for a given input is critical to ensuring high-quality outputs. Chroma provides an efficient way to store and retrieve examples based on semantic similarity, enabling dynamic example selection in workflows.

1. **Embedding and Vector Store Initialization**

   - Use OpenAIEmbeddings to embed examples.
   - Store the embeddings in a Chroma vector store for efficient retrieval.

2. **Example Storage**

   - Examples are stored with both their content and metadata.
   - Metadata can include details such as the question and answer, which are used for further processing after retrieval.

3. **Similarity Search**

   - Query the vector store to retrieve the most similar examples based on the input.
   - Enables context-aware dynamic prompt creation.

In [26]:
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import ChatOpenAI

# Initialize embeddings and vector store
embeddings = OpenAIEmbeddings()
chroma = Chroma(persist_directory="example_selector", embedding_function=embeddings)

# Define examples for Few-Shot prompting
examples = [
    {"question": "Who lived longer, Steve Jobs or Einstein?", "answer": "Einstein"},
    {"question": "When was the founder of Naver born?", "answer": "June 22, 1967"},
    {"question": "Who was the reigning king when Yulgok Yi's mother was born?", "answer": "Yeonsangun"},
    {"question": "Are the directors of Oldboy and Parasite from the same country?", "answer": "Yes"},
]

# Add examples to the vector store
texts = [example["question"] for example in examples]
metadatas = [example for example in examples]
chroma.add_texts(texts=texts, metadatas=metadatas)

# Set up Example Selector
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=chroma,  # Only vectorstore is needed
    k=1  # Number of examples to select
)

# Define Few-Shot Prompt Template
example_prompt_template = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}\n"
)
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt_template,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

# Query input and process
query = {"question": "How old was Bill Gates when Google was founded?"}
formatted_prompt = prompt.format(**query)

# Print the constructed prompt
print("Constructed Prompt:\n")
print(formatted_prompt)

# Initialize the language model
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Query the language model with the constructed prompt
response = llm.predict(formatted_prompt)
print("\nResponse:\n")
print(response)


Constructed Prompt:

Question:
Who lived longer, Steve Jobs or Einstein?
Answer:
Einstein


Question:
How old was Bill Gates when Google was founded?
Answer:

Response:

Google was founded on September 4, 1998. Bill Gates was born on October 28, 1955, which means he was 42 years old at the time Google was founded.


In [17]:
# Step 4: Set up Example Selector
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embedding_function=embeddings,
    vector_store=chroma,
    k=1,  # Select the top 1 most similar example
)

# Step 5: Define Few-Shot Prompt Template
example_prompt_template = "Question:\n{question}\nAnswer:\n{answer}\n"
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt_template,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

TypeError: SemanticSimilarityExampleSelector.from_examples() missing 2 required positional arguments: 'embeddings' and 'vectorstore_cls'

In [35]:
# Print the most similar example
print(f"Most similar example to the input:\n{query}\n")
for result in results:
    print(f'question:\n{result.page_content}')
    print(f'answer:\n{result.metadata["answer"]}')

Most similar example to the input:
How old was Bill Gates when Google was founded?

question:
Who lived longer, Steve Jobs or Einstein?
answer:
Einstein


## FewShotChatMessagePromptTemplate

The `FewShotChatMessagePromptTemplate` leverages a few-shot learning approach to dynamically generate chat-based prompts by combining relevant examples with a structured format. This method is especially effective for tasks like summarization, meeting minute creation, or document processing.


In [36]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Initialize the Vector DB
chroma = Chroma(persist_directory="fewshot_chat", embedding_function=embeddings)

# Define examples for few-shot prompting
examples = [
    {
        "instruction": "Please summarize the following document",
        "input": "This document discusses the importance of teamwork in corporate settings, emphasizing collaboration and communication.",
        "answer": "Teamwork is essential in corporate settings, focusing on collaboration and communication.",
    },
    {
        "instruction": "Please write the meeting minutes",
        "input": "On December 26, 2023, the product development team held a weekly meeting. The meeting focused on progress updates and milestone planning.",
        "answer": "Meeting minutes: Weekly meeting held on December 26, 2023. Focused on progress updates and milestone planning.",
    },
]

# Add examples to the vector store
texts = [example["instruction"] + " " + example["input"] for example in examples]
metadatas = examples
chroma.add_texts(texts=texts, metadatas=metadatas)

# Example query
query = {
    "instruction": "Please write the meeting minutes",
    "input": "On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.",
}

# Perform similarity search
query_text = query["instruction"] + " " + query["input"]
results = chroma.similarity_search(query_text, k=1)
print(results)

[Document(metadata={'answer': 'Meeting minutes: Weekly meeting held on December 26, 2023. Focused on progress updates and milestone planning.', 'input': 'On December 26, 2023, the product development team held a weekly meeting. The meeting focused on progress updates and milestone planning.', 'instruction': 'Please write the meeting minutes'}, page_content='Please write the meeting minutes On December 26, 2023, the product development team held a weekly meeting. The meeting focused on progress updates and milestone planning.')]


In [37]:
# Print the most similar example
print(f"Most similar example to the input:\n{query_text}\n")
for result in results:
    print(f'Instruction:\n{result.metadata["instruction"]}')
    print(f'Input:\n{result.metadata["input"]}')
    print(f'Answer:\n{result.metadata["answer"]}')

Most similar example to the input:
Please write the meeting minutes On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.

Instruction:
Please write the meeting minutes
Input:
On December 26, 2023, the product development team held a weekly meeting. The meeting focused on progress updates and milestone planning.
Answer:
Meeting minutes: Weekly meeting held on December 26, 2023. Focused on progress updates and milestone planning.


In [38]:
# Create the final prompt template
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        (
            "human",
            "{instruction}:\n{input}",
        ),
    ]
)

# Combine the prompt and the best example
best_example = results[0].metadata
filled_prompt = final_prompt.format_messages(**best_example)

# Print the filled prompt
print("\nFilled Prompt:\n")
for message in filled_prompt:
    # Determine message type and extract content
    message_type = type(message).__name__  # e.g., SystemMessage, HumanMessage, AIMessage
    content = message.content
    print(f"{message_type}:\n{content}\n")


Filled Prompt:

SystemMessage:
You are a helpful assistant.

HumanMessage:
Please write the meeting minutes:
On December 26, 2023, the product development team held a weekly meeting. The meeting focused on progress updates and milestone planning.

AIMessage:
Meeting minutes: Weekly meeting held on December 26, 2023. Focused on progress updates and milestone planning.



### Resolving Similarity Search Issues in Example Selector

When calculating similarity, both `instruction` and `input` are used. However, searching based only on `instruction` does not yield accurate similarity results.

To resolve this, a custom class for similarity calculation is defined.

Below is an example of incorrectly retrieved results.


In [39]:
# Initialize example selector
example_selector = chroma

# Example question with only instruction
question = {
    "instruction": "Please write the meeting minutes",
}

# Select examples using similarity search
query_text = question["instruction"]
results = example_selector.similarity_search(query_text, k=1)

# Print the selected examples
print("Most similar examples:")
for result in results:
    print(f'Instruction: {result.metadata["instruction"]}')
    print(f'Input: {result.metadata["input"]}')
    print(f'Answer: {result.metadata["answer"]}')

Most similar examples:
Instruction: Please write the meeting minutes
Input: On December 26, 2023, the product development team held a weekly meeting. The meeting focused on progress updates and milestone planning.
Answer: Meeting minutes: Weekly meeting held on December 26, 2023. Focused on progress updates and milestone planning.


In [40]:
# Example question with only instruction
question = {
    "instruction": "Please write the meeting minutes",
}

# Use similarity_search to find the most similar example
query_text = question["instruction"]
results = chroma.similarity_search(query_text, k=1)

# Print the selected example
print("Most similar example:")
for result in results:
    print(f'Instruction: {result.metadata["instruction"]}')
    print(f'Input: {result.metadata["input"]}')
    print(f'Answer: {result.metadata["answer"]}')

Most similar example:
Instruction: Please write the meeting minutes
Input: On December 26, 2023, the product development team held a weekly meeting. The meeting focused on progress updates and milestone planning.
Answer: Meeting minutes: Weekly meeting held on December 26, 2023. Focused on progress updates and milestone planning.


In [41]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain_chroma import Chroma

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Initialize the Vector DB
chroma = Chroma(persist_directory="fewshot_chat", embedding_function=embeddings)

# Define examples for few-shot prompting
examples = [
    {
        "instruction": "Please summarize the following document",
        "input": "This document discusses the importance of teamwork in corporate settings, emphasizing collaboration and communication.",
        "answer": "Teamwork is essential in corporate settings, focusing on collaboration and communication.",
    },
    {
        "instruction": "Please write the meeting minutes",
        "input": "On December 26, 2023, the product development team held a weekly meeting. The meeting focused on progress updates and milestone planning.",
        "answer": "Meeting minutes: Weekly meeting held on December 26, 2023. Focused on progress updates and milestone planning.",
    },
    {
        "instruction": "Please correct the following sentences",
        "input": "The company expects revenue to increase this year. The new strategy is working well.",
        "answer": "The company expects its revenue to increase this year as the new strategy is working well.",
    },
]

# Add examples to the vector store
texts = [example["instruction"] + " " + example["input"] for example in examples]
metadatas = examples
chroma.add_texts(texts=texts, metadatas=metadatas)

# Example question to test
questions = [
    {
        "instruction": "Please write the meeting minutes",
        "input": "On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.",
    },
    {
        "instruction": "Please summarize the document",
        "input": "This document is a detailed 30-page report on 'Global Economic Outlook for 2023'. The report comprehensively covers the importance of sustainable urban development, current urbanization issues, and various strategies to make urban development sustainable. It also introduces successful sustainable urban development cases from multiple countries and summarizes the lessons learned from these cases.",
    },
    {
        "instruction": "Please correct the following sentences",
        "input": "The company expects revenue to increase this year. The new strategy is working well.",
    },
]

# Process each question and find the most similar example
for question in questions:
    query_text = question["instruction"] + " " + question["input"]
    results = chroma.similarity_search(query_text, k=1)

    print(f"Question:\n{query_text}\n")
    print("Most similar example:")
    for result in results:
        print(f'Instruction: {result.metadata["instruction"]}')
        print(f'Input: {result.metadata["input"]}')
        print(f'Answer: {result.metadata["answer"]}')
    print("\n---\n")


Question:
Please write the meeting minutes On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.

Most similar example:
Instruction: Please write the meeting minutes
Input: On December 26, 2023, the product development team held a weekly meeting. The meeting focused on progress updates and milestone planning.
Answer: Meeting minutes: Weekly meeting held on December 26, 2023. Focused on progress updates and milestone planning.

---

Question:
Please summarize the document This document is a detailed 30-page report on 'Global Economic Outlook 

In [42]:
# Example question with only instruction
question = {
    "instruction": "Please summarize the following sentence",
}

# Use similarity_search to find the most similar example
query_text = question["instruction"]
results = chroma.similarity_search(query_text, k=1)

# Print the selected example
print("Most similar example:")
for result in results:
    print(f'Instruction: {result.metadata["instruction"]}')
    print(f'Input: {result.metadata["input"]}')
    print(f'Answer: {result.metadata["answer"]}')

Most similar example:
Instruction: Please summarize the following document
Input: This document discusses the importance of teamwork in corporate settings, emphasizing collaboration and communication.
Answer: Teamwork is essential in corporate settings, focusing on collaboration and communication.


In [43]:
# Use Chroma for example selection
def select_examples(query):
    query_text = query["instruction"] + " " + query.get("input", "")
    results = chroma.similarity_search(query_text, k=1)
    return results


# Example selector using Chroma
def custom_selector(query):
    results = select_examples(query)
    selected_examples = [
        {
            "instruction": result.metadata["instruction"],
            "input": result.metadata["input"],
            "answer": result.metadata["answer"],
        }
        for result in results
    ]
    return selected_examples


# Define a new example prompt template with the custom selector
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

# Create a prompt template with selected examples
def create_fewshot_prompt(query):
    selected_examples = custom_selector(query)
    fewshot_prompt = ""
    for example in selected_examples:
        fewshot_prompt += example_prompt.format(
            instruction=example["instruction"],
            input=example["input"],
            answer=example["answer"],
        )
    return fewshot_prompt

# Create the final prompt for the chain
def create_final_prompt(query):
    fewshot_prompt = create_fewshot_prompt(query)
    final_prompt = (
        "You are a helpful assistant.\n\n" + fewshot_prompt + "\n\n"
        f"Human:\n{query['instruction']}:\n{query['input']}\nAI:"
    )
    return final_prompt

# Example questions
questions = [
    {
        "instruction": "Please write the meeting minutes",
        "input": "On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.",
    },
    {
        "instruction": "Please summarize the document",
        "input": "This document is a detailed 30-page report on 'Global Economic Outlook for 2023'. The report comprehensively covers the importance of sustainable urban development, current urbanization issues, and various strategies to make urban development sustainable. It also introduces successful sustainable urban development cases from multiple countries and summarizes the lessons learned from these cases.",
    },
    {
        "instruction": "Please correct the following sentences",
        "input": "The company expects revenue to increase this year. The new strategy is working well.",
    },
]

# Process each question and print the responses
for question in questions:
    final_prompt = create_final_prompt(question)
    print("\nFinal Prompt:\n")
    print(final_prompt)
    print("\n---\n")


Final Prompt:

You are a helpful assistant.

Human: Please write the meeting minutes:
On December 26, 2023, the product development team held a weekly meeting. The meeting focused on progress updates and milestone planning.
AI: Meeting minutes: Weekly meeting held on December 26, 2023. Focused on progress updates and milestone planning.

Human:
Please write the meeting minutes:
On December 26, 2023, the product development team of ABC Technology Company held a weekly progress meeting for a new mobile application project. The meeting was attended by Project Manager Choi Hyun-soo, Lead Developer Hwang Ji-yeon, and UI/UX Designer Kim Tae-young. The main purpose of the meeting was to review the current progress of the project and establish plans for upcoming milestones. Each team member provided updates on their respective areas of work, and the team set goals for the next week.
AI:

---


Final Prompt:

You are a helpful assistant.

Human: Please summarize the following document:
This do